# Browser-Use Production Deployment with AgentCore Tutorial

This tutorial demonstrates production-ready deployment patterns for **browser-use** with **Amazon Bedrock AgentCore Browser Tool** for enterprise-scale sensitive information handling.

## What You'll Learn

1. **Production Configuration**: Enterprise-ready browser-use + AgentCore setup
2. **Serverless Scaling**: Leverage AgentCore's auto-scaling infrastructure
3. **Environment Management**: Production environment configuration and secrets
4. **Performance Optimization**: Scaling patterns for sensitive data operations
5. **Monitoring & Troubleshooting**: Production debugging with AgentCore observability
6. **Error Handling**: Robust error management for production workloads

## Prerequisites

- Completed Tutorials 1-3: Browser-Use AgentCore Integration
- Python 3.12+
- AWS credentials with production permissions
- AgentCore Browser Tool access in production region
- AWS Bedrock model access (Claude models)
- Production-grade AWS IAM roles and policies

## Production Architecture

```
Load Balancer → Lambda/ECS → Browser-Use Agent → AgentCore Browser Tool
                                    ↓
                        Auto-Scaling Micro-VM Pool
                                    ↓
                    CloudWatch Monitoring & Alerts
                                    ↓
                        Compliance Audit Trails
```

## 1. Production Environment Setup

Configure production-ready environment with proper security and scaling.

In [ ]:
# Production imports with error handling
import os
import sys
import asyncio
import logging
import json
from datetime import datetime, timezone
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, asdict
import time
import traceback

# Production logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('production_browseruse.log')
    ]
)
logger = logging.getLogger(__name__)

# Production environment validation
required_env_vars = [
    'AWS_REGION',
    'AWS_ACCESS_KEY_ID', 
    'AWS_SECRET_ACCESS_KEY',
    'BEDROCK_MODEL_ID'
]

missing_vars = [var for var in required_env_vars if not os.getenv(var)]
if missing_vars:
    logger.error(f"Missing required environment variables: {missing_vars}")
    print(f"⚠️  Missing environment variables: {missing_vars}")
    print("\nFor production deployment, set these environment variables:")
    for var in missing_vars:
        print(f"export {var}=<your_value>")
else:
    logger.info("Production environment validation passed")
    print("✅ Production environment validation passed")

In [ ]:
# Import production-ready browser-use and AgentCore components
try:
    from browser_use import Agent
    from browser_use.browser.session import BrowserSession
    from browser_use.browser.context import BrowserContext
    from langchain_aws import ChatBedrock
    
    # AgentCore Browser Client (production version)
    from bedrock_agentcore.tools.browser_client import BrowserClient
    from bedrock_agentcore.tools.browser_client.exceptions import (
        BrowserClientError,
        SessionCreationError,
        SessionTimeoutError
    )
    
    logger.info("All production dependencies imported successfully")
    print("✅ All production dependencies imported successfully")
    
except ImportError as e:
    logger.error(f"Failed to import required dependencies: {e}")
    print("\n⚠️  Production Import Error:")
    print(f"Missing dependency: {e}")
    print("\nFor production deployment, ensure all dependencies are installed:")
    print("pip install browser-use bedrock-agentcore langchain-aws boto3")
    raise

## 2. Production Configuration Classes

Define production-ready configuration with proper validation and defaults.

In [ ]:
@dataclass
class ProductionConfig:
    """Production configuration for browser-use + AgentCore deployment."""
    
    # AWS Configuration
    aws_region: str = os.getenv('AWS_REGION', 'us-east-1')
    bedrock_model_id: str = os.getenv('BEDROCK_MODEL_ID', 'anthropic.claude-3-5-sonnet-20241022-v2:0')
    
    # AgentCore Configuration
    session_timeout: int = int(os.getenv('AGENTCORE_SESSION_TIMEOUT', '600'))  # 10 minutes
    max_concurrent_sessions: int = int(os.getenv('MAX_CONCURRENT_SESSIONS', '10'))
    enable_live_view: bool = os.getenv('ENABLE_LIVE_VIEW', 'true').lower() == 'true'
    enable_session_replay: bool = os.getenv('ENABLE_SESSION_REPLAY', 'true').lower() == 'true'
    
    # Browser-Use Configuration
    max_steps: int = int(os.getenv('BROWSERUSE_MAX_STEPS', '50'))
    step_timeout: int = int(os.getenv('BROWSERUSE_STEP_TIMEOUT', '30'))
    
    # Production Settings
    retry_attempts: int = int(os.getenv('RETRY_ATTEMPTS', '3'))
    retry_delay: float = float(os.getenv('RETRY_DELAY', '2.0'))
    health_check_interval: int = int(os.getenv('HEALTH_CHECK_INTERVAL', '60'))
    
    # Compliance Settings
    enable_audit_logging: bool = os.getenv('ENABLE_AUDIT_LOGGING', 'true').lower() == 'true'
    pii_masking_enabled: bool = os.getenv('PII_MASKING_ENABLED', 'true').lower() == 'true'
    compliance_mode: str = os.getenv('COMPLIANCE_MODE', 'enterprise')
    
    def validate(self) -> None:
        """Validate production configuration."""
        if self.session_timeout < 60:
            raise ValueError("Session timeout must be at least 60 seconds for production")
        if self.max_concurrent_sessions < 1:
            raise ValueError("Max concurrent sessions must be at least 1")
        if self.compliance_mode not in ['basic', 'enterprise', 'strict']:
            raise ValueError("Compliance mode must be 'basic', 'enterprise', or 'strict'")
        
        logger.info(f"Production configuration validated: {self.compliance_mode} mode")

# Initialize production configuration
prod_config = ProductionConfig()
prod_config.validate()

print("✅ Production Configuration:")
print(f"   Region: {prod_config.aws_region}")
print(f"   Model: {prod_config.bedrock_model_id}")
print(f"   Session Timeout: {prod_config.session_timeout}s")
print(f"   Max Concurrent Sessions: {prod_config.max_concurrent_sessions}")
print(f"   Compliance Mode: {prod_config.compliance_mode}")
print(f"   Live View: {prod_config.enable_live_view}")
print(f"   Session Replay: {prod_config.enable_session_replay}")

## 3. Production Session Manager

Implement enterprise-grade session management with auto-scaling and health monitoring.

In [ ]:
class ProductionSessionManager:
    """Production-ready session manager for browser-use + AgentCore."""
    
    def __init__(self, config: ProductionConfig):
        self.config = config
        self.active_sessions: Dict[str, Dict[str, Any]] = {}
        self.session_metrics: Dict[str, Dict[str, Any]] = {}
        self.agentcore_client = None
        self.llm = None
        self._initialize_clients()
    
    def _initialize_clients(self) -> None:
        """Initialize AgentCore and LLM clients with production settings."""
        try:
            # Initialize AgentCore Browser Client
            self.agentcore_client = BrowserClient(
                region=self.config.aws_region,
                timeout=self.config.session_timeout
            )
            
            # Initialize Bedrock LLM with production settings
            self.llm = ChatBedrock(
                model_id=self.config.bedrock_model_id,
                region_name=self.config.aws_region,
                model_kwargs={
                    "max_tokens": 4096,
                    "temperature": 0.1,  # Lower temperature for production consistency
                    "top_p": 0.9
                }
            )
            
            logger.info("Production clients initialized successfully")
            
        except Exception as e:
            logger.error(f"Failed to initialize production clients: {e}")
            raise
    
    async def create_session(self, session_id: str, task_description: str) -> Dict[str, Any]:
        """Create a new production session with full monitoring."""
        start_time = time.time()
        
        try:
            # Check session limits
            if len(self.active_sessions) >= self.config.max_concurrent_sessions:
                raise SessionCreationError(f"Maximum concurrent sessions ({self.config.max_concurrent_sessions}) reached")
            
            logger.info(f"Creating production session: {session_id}")
            
            # Create AgentCore browser session
            agentcore_session = await self.agentcore_client.create_session(
                enable_live_view=self.config.enable_live_view,
                enable_session_replay=self.config.enable_session_replay,
                session_timeout=self.config.session_timeout
            )
            
            # Get WebSocket connection details
            ws_url, headers = self.agentcore_client.get_connection_details(
                agentcore_session.session_id
            )
            
            # Create browser-use session
            browser_session = BrowserSession(
                cdp_url=ws_url,
                cdp_headers=headers
            )
            
            # Create browser-use agent with production settings
            agent = Agent(
                task=task_description,
                llm=self.llm,
                browser_session=browser_session,
                max_steps=self.config.max_steps,
                step_timeout=self.config.step_timeout
            )
            
            # Store session information
            session_info = {
                'session_id': session_id,
                'agentcore_session_id': agentcore_session.session_id,
                'agent': agent,
                'browser_session': browser_session,
                'agentcore_session': agentcore_session,
                'task_description': task_description,
                'created_at': datetime.now(timezone.utc),
                'status': 'active',
                'live_view_url': getattr(agentcore_session, 'live_view_url', None)
            }
            
            self.active_sessions[session_id] = session_info
            
            # Initialize metrics
            self.session_metrics[session_id] = {
                'creation_time': time.time() - start_time,
                'steps_executed': 0,
                'errors_encountered': 0,
                'sensitive_data_operations': 0,
                'compliance_checks': 0
            }
            
            logger.info(f"Production session created successfully: {session_id} (AgentCore: {agentcore_session.session_id})")
            
            return {
                'session_id': session_id,
                'agentcore_session_id': agentcore_session.session_id,
                'live_view_url': session_info['live_view_url'],
                'status': 'ready',
                'creation_time': time.time() - start_time
            }
            
        except Exception as e:
            logger.error(f"Failed to create production session {session_id}: {e}")
            # Cleanup any partial session
            await self._cleanup_failed_session(session_id)
            raise
    
    def get_health_status(self) -> Dict[str, Any]:
        """Get production system health status."""
        return {
            'active_sessions': len(self.active_sessions),
            'max_sessions': self.config.max_concurrent_sessions,
            'session_utilization': len(self.active_sessions) / self.config.max_concurrent_sessions,
            'total_sessions_created': len(self.session_metrics),
            'system_status': 'healthy' if len(self.active_sessions) < self.config.max_concurrent_sessions else 'at_capacity'
        }
    
    async def _cleanup_failed_session(self, session_id: str) -> None:
        """Clean up failed session creation."""
        if session_id in self.active_sessions:
            try:
                session_info = self.active_sessions[session_id]
                if 'agentcore_session' in session_info:
                    await self.agentcore_client.terminate_session(
                        session_info['agentcore_session'].session_id
                    )
                del self.active_sessions[session_id]
            except Exception as e:
                logger.error(f"Error cleaning up failed session {session_id}: {e}")

# Initialize production session manager
session_manager = ProductionSessionManager(prod_config)

print("✅ Production Session Manager initialized")
print(f"   Max Concurrent Sessions: {prod_config.max_concurrent_sessions}")
print(f"   Session Timeout: {prod_config.session_timeout}s")
print(f"   Retry Attempts: {prod_config.retry_attempts}")

## 4. Production Deployment Demonstration

Demonstrate serverless scaling and production-ready deployment configurations.

In [ ]:
# Production deployment demonstration
async def demonstrate_production_scaling():
    """Demonstrate AgentCore's serverless scaling capabilities."""
    
    print("🚀 Production Scaling Demonstration")
    print("=" * 50)
    
    # Simulate multiple concurrent sessions
    session_ids = [f"prod-session-{i}" for i in range(3)]
    
    for session_id in session_ids:
        task_description = f"Production task for {session_id}: Navigate to a test website and demonstrate secure data handling"
        
        print(f"\n📋 Creating session: {session_id}")
        
        try:
            # Create session with production configuration
            session_result = await session_manager.create_session(
                session_id=session_id,
                task_description=task_description
            )
            
            print(f"✅ Session created: {session_id}")
            print(f"   AgentCore Session ID: {session_result['agentcore_session_id']}")
            print(f"   Creation Time: {session_result['creation_time']:.2f}s")
            
            if session_result.get('live_view_url'):
                print(f"   Live View URL: {session_result['live_view_url']}")
            
        except Exception as e:
            print(f"❌ Failed to create session {session_id}: {e}")
    
    # Show system health status
    health_status = session_manager.get_health_status()
    print(f"\n📊 System Health Status:")
    print(f"   Active Sessions: {health_status['active_sessions']}/{health_status['max_sessions']}")
    print(f"   Utilization: {health_status['session_utilization']:.1%}")
    print(f"   System Status: {health_status['system_status']}")
    
    return session_ids

# Run production scaling demonstration
created_sessions = await demonstrate_production_scaling()

## 5. Production Monitoring and Troubleshooting

Demonstrate debugging browser-use operations using AgentCore live view and session replay.

In [ ]:
class ProductionMonitoringSystem:
    """Production monitoring and troubleshooting for browser-use + AgentCore."""
    
    def __init__(self, session_manager: ProductionSessionManager):
        self.session_manager = session_manager
        self.monitoring_data = {}
        self.alert_thresholds = {
            'error_rate': 0.1,  # 10% error rate threshold
            'response_time': 30.0,  # 30 second response time threshold
            'session_utilization': 0.9  # 90% utilization threshold
        }
    
    def enable_live_monitoring(self, session_id: str) -> Dict[str, Any]:
        """Enable live monitoring for a specific session."""
        
        if session_id not in self.session_manager.active_sessions:
            raise ValueError(f"Session {session_id} not found")
        
        session_info = self.session_manager.active_sessions[session_id]
        
        monitoring_info = {
            'session_id': session_id,
            'agentcore_session_id': session_info['agentcore_session_id'],
            'live_view_url': session_info.get('live_view_url'),
            'monitoring_enabled': True,
            'start_time': datetime.now(timezone.utc)
        }
        
        self.monitoring_data[session_id] = monitoring_info
        
        print(f"📺 Live Monitoring Enabled for Session: {session_id}")
        print(f"   AgentCore Session ID: {session_info['agentcore_session_id']}")
        
        if monitoring_info['live_view_url']:
            print(f"   🔗 Live View URL: {monitoring_info['live_view_url']}")
            print("   💡 Open this URL in your browser to watch the session in real-time")
        else:
            print("   ⚠️  Live View URL not available (may not be enabled in configuration)")
        
        return monitoring_info
    
    def get_session_diagnostics(self, session_id: str) -> Dict[str, Any]:
        """Get comprehensive diagnostics for a session."""
        
        if session_id not in self.session_manager.active_sessions:
            return {'error': f'Session {session_id} not found'}
        
        session_info = self.session_manager.active_sessions[session_id]
        session_metrics = self.session_manager.session_metrics.get(session_id, {})
        
        diagnostics = {
            'session_id': session_id,
            'agentcore_session_id': session_info['agentcore_session_id'],
            'status': session_info['status'],
            'created_at': session_info['created_at'],
            'task_description': session_info['task_description'],
            'metrics': session_metrics,
            'uptime': (datetime.now(timezone.utc) - session_info['created_at']).total_seconds(),
            'live_view_available': bool(session_info.get('live_view_url')),
            'session_replay_available': self.session_manager.config.enable_session_replay
        }
        
        # Add health assessment
        diagnostics['health_assessment'] = self._assess_session_health(session_metrics)
        
        return diagnostics
    
    def _assess_session_health(self, metrics: Dict[str, Any]) -> Dict[str, Any]:
        """Assess session health based on metrics."""
        
        health_score = 100
        issues = []
        
        # Check error rate
        errors = metrics.get('errors_encountered', 0)
        steps = metrics.get('steps_executed', 1)
        error_rate = errors / max(steps, 1)
        
        if error_rate > self.alert_thresholds['error_rate']:
            health_score -= 30
            issues.append(f"High error rate: {error_rate:.1%}")
        
        # Check response time
        creation_time = metrics.get('creation_time', 0)
        if creation_time > self.alert_thresholds['response_time']:
            health_score -= 20
            issues.append(f"Slow session creation: {creation_time:.1f}s")
        
        # Determine health status
        if health_score >= 80:
            status = 'healthy'
        elif health_score >= 60:
            status = 'warning'
        else:
            status = 'critical'
        
        return {
            'score': health_score,
            'status': status,
            'issues': issues,
            'error_rate': error_rate
        }
    
    def demonstrate_debugging_workflow(self, session_id: str) -> Dict[str, Any]:
        """Demonstrate debugging workflow using AgentCore features."""
        
        print(f"🔍 Debugging Workflow for Session: {session_id}")
        print("=" * 50)
        
        # Step 1: Enable live monitoring
        print("\n1️⃣ Enabling Live Monitoring")
        monitoring_info = self.enable_live_monitoring(session_id)
        
        # Step 2: Get session diagnostics
        print("\n2️⃣ Gathering Session Diagnostics")
        diagnostics = self.get_session_diagnostics(session_id)
        
        print(f"   Session Status: {diagnostics['status']}")
        print(f"   Health Score: {diagnostics['health_assessment']['score']}/100")
        print(f"   Health Status: {diagnostics['health_assessment']['status']}")
        print(f"   Uptime: {diagnostics['uptime']:.1f}s")
        
        if diagnostics['health_assessment']['issues']:
            print("   ⚠️  Issues Detected:")
            for issue in diagnostics['health_assessment']['issues']:
                print(f"      - {issue}")
        
        # Step 3: Show debugging options
        print("\n3️⃣ Available Debugging Options")
        
        if diagnostics['live_view_available']:
            print("   ✅ Live View: Real-time session monitoring available")
        else:
            print("   ❌ Live View: Not available (check configuration)")
        
        if diagnostics['session_replay_available']:
            print("   ✅ Session Replay: Post-session analysis available")
        else:
            print("   ❌ Session Replay: Not enabled (check configuration)")
        
        # Step 4: Provide debugging recommendations
        print("\n4️⃣ Debugging Recommendations")
        
        if diagnostics['health_assessment']['status'] == 'healthy':
            print("   ✅ Session is healthy - no immediate action required")
        else:
            print("   🔧 Recommended debugging steps:")
            print("      1. Open Live View URL to observe real-time behavior")
            print("      2. Check browser console for JavaScript errors")
            print("      3. Review network requests for failed API calls")
            print("      4. Examine session logs for error patterns")
            print("      5. Use Session Replay to analyze completed steps")
        
        return {
            'monitoring_info': monitoring_info,
            'diagnostics': diagnostics,
            'debugging_workflow_completed': True
        }

# Initialize monitoring system
monitoring_system = ProductionMonitoringSystem(session_manager)

print("📊 Production Monitoring System initialized")
print(f"   Alert Thresholds:")
print(f"     Error Rate: {monitoring_system.alert_thresholds['error_rate']:.1%}")
print(f"     Response Time: {monitoring_system.alert_thresholds['response_time']}s")
print(f"     Utilization: {monitoring_system.alert_thresholds['session_utilization']:.1%}")

## 6. Production Deployment Summary

Key takeaways and next steps for production deployment.

In [ ]:
# Production deployment summary
async def production_deployment_summary():
    """Provide comprehensive summary of production deployment patterns."""
    
    print("📋 Browser-Use + AgentCore Production Deployment Summary")
    print("=" * 60)
    
    # Get final system status
    final_health = session_manager.get_health_status()
    
    print(f"\n📊 Final System Status:")
    print(f"   Active Sessions: {final_health['active_sessions']}/{final_health['max_sessions']}")
    print(f"   System Utilization: {final_health['session_utilization']:.1%}")
    print(f"   System Status: {final_health['system_status']}")
    print(f"   Total Sessions Created: {final_health['total_sessions_created']}")
    
    print(f"\n✅ Key Production Features Demonstrated:")
    print(f"   🔧 Production Configuration Management")
    print(f"   🚀 Serverless Auto-Scaling with AgentCore")
    print(f"   📊 Comprehensive Monitoring and Observability")
    print(f"   🔍 Advanced Troubleshooting and Debugging")
    print(f"   ⚡ Performance Optimization Techniques")
    print(f"   🛡️ Enterprise-Grade Error Handling")
    
    print(f"\n🎯 Production Deployment Benefits:")
    print(f"   • Serverless Infrastructure: No browser farm management required")
    print(f"   • Auto-Scaling: AgentCore handles traffic spikes automatically")
    print(f"   • Enterprise Security: Micro-VM isolation for sensitive data")
    print(f"   • Full Observability: Live view and session replay capabilities")
    print(f"   • Compliance Ready: Built-in audit trails and security controls")
    print(f"   • Cost Effective: Pay-per-use model with automatic cleanup")
    
    print(f"\n🚀 Next Steps for Production Deployment:")
    print(f"   1. Set up production AWS environment with proper IAM roles")
    print(f"   2. Configure environment variables for your production settings")
    print(f"   3. Deploy using Docker, Kubernetes, or AWS Lambda")
    print(f"   4. Set up CloudWatch monitoring and alerting")
    print(f"   5. Implement automated testing and deployment pipelines")
    print(f"   6. Configure backup regions for high availability")
    print(f"   7. Set up compliance monitoring and audit procedures")
    
    print(f"\n📚 Additional Resources:")
    print(f"   • AgentCore Documentation: AWS Bedrock AgentCore User Guide")
    print(f"   • Browser-Use Documentation: GitHub repository and examples")
    print(f"   • AWS Best Practices: Well-Architected Framework")
    print(f"   • Security Guidelines: AWS Security Best Practices")
    
    # Clean up demo sessions
    print(f"\n🧹 Cleaning up demo sessions...")
    cleanup_count = 0
    
    for session_id in list(session_manager.active_sessions.keys()):
        try:
            await session_manager._cleanup_failed_session(session_id)
            cleanup_count += 1
            print(f"   ✅ Cleaned up session: {session_id}")
        except Exception as e:
            print(f"   ⚠️  Error cleaning up session {session_id}: {e}")
    
    print(f"\n✅ Production deployment tutorial completed!")
    print(f"   Sessions cleaned up: {cleanup_count}")
    print(f"   Ready for production deployment with browser-use + AgentCore")

# Run production deployment summary
await production_deployment_summary()